In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
import os
from tqdm import tqdm

import numpy as np
import torch

from zerogercrnn.lib.train.run import TrainEpochRunner

from zerogercrnn.experiments.js.ast_level.pdata import SourceFile, DataProvider, DataReader
from zerogercrnn.experiments.js.ast_level.data import ENCODING

In [4]:
DIR_DATASET = '/Users/zerogerc/Documents/datasets/js_dataset.tar/processed'

FILE_TRAINING = os.path.join(DIR_DATASET, 'programs_training_one_hot.json')
FILE_EVAL = os.path.join(DIR_DATASET, 'programs_eval_one_hot.json')

SEQ_LEN = 50
BATCH_SIZE = 80

In [15]:
reader = DataReader(
    file_training=FILE_TRAINING,
    file_eval=FILE_EVAL,
    encoding=ENCODING
)

  0%|          | 0/100000 [00:00<?, ?it/s]

Parsing Train


  0%|          | 73/50000 [00:00<01:08, 729.22it/s]

Parsing Eval


100%|██████████| 50000/50000 [02:01<00:00, 412.56it/s]


In [16]:
provider = DataProvider(
    data_reader=reader,
    seq_len=SEQ_LEN,
    batch_size=BATCH_SIZE
)

100%|██████████| 50000/50000 [00:01<00:00, 26769.92it/s]


In [31]:
import time

start = time.time()
N, T = next(provider.get_batched_epoch())
end = time.time()
print((end - start) * 1000)

3.896951675415039
